# Table Design and Load

In this lab, you will use a set of eight tables based on the TPC Benchmark data model. You create these tables within your Redshift cluster then load these tables with sample data stored in S3.

## Create Table

To create 8 tables from TPC Benchmark data model, run create table statements. Given below is the data model for the tables.

![model](https://user-images.githubusercontent.com/62965911/225864429-ee5f4dec-0a56-4d0c-855e-8fffa122c5c2.png)

In [ ]:
DROP TABLE IF EXISTS partsupp;
DROP TABLE IF EXISTS lineitem;
DROP TABLE IF EXISTS supplier;
DROP TABLE IF EXISTS part;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS customer;
DROP TABLE IF EXISTS nation;
DROP TABLE IF EXISTS region;

CREATE TABLE region (
  R_REGIONKEY bigint NOT NULL,
  R_NAME varchar(25),
  R_COMMENT varchar(152))
diststyle all;

CREATE TABLE nation (
  N_NATIONKEY bigint NOT NULL,
  N_NAME varchar(25),
  N_REGIONKEY bigint,
  N_COMMENT varchar(152))
diststyle all;

create table customer (
  C_CUSTKEY bigint NOT NULL,
  C_NAME varchar(25),
  C_ADDRESS varchar(40),
  C_NATIONKEY bigint,
  C_PHONE varchar(15),
  C_ACCTBAL decimal(18,4),
  C_MKTSEGMENT varchar(10),
  C_COMMENT varchar(117))
diststyle all;

create table orders (
  O_ORDERKEY bigint NOT NULL,
  O_CUSTKEY bigint,
  O_ORDERSTATUS varchar(1),
  O_TOTALPRICE decimal(18,4),
  O_ORDERDATE Date,
  O_ORDERPRIORITY varchar(15),
  O_CLERK varchar(15),
  O_SHIPPRIORITY Integer,
  O_COMMENT varchar(79))
distkey (O_ORDERKEY)
sortkey (O_ORDERDATE);

create table part (
  P_PARTKEY bigint NOT NULL,
  P_NAME varchar(55),
  P_MFGR  varchar(25),
  P_BRAND varchar(10),
  P_TYPE varchar(25),
  P_SIZE integer,
  P_CONTAINER varchar(10),
  P_RETAILPRICE decimal(18,4),
  P_COMMENT varchar(23))
diststyle all;

create table supplier (
  S_SUPPKEY bigint NOT NULL,
  S_NAME varchar(25),
  S_ADDRESS varchar(40),
  S_NATIONKEY bigint,
  S_PHONE varchar(15),
  S_ACCTBAL decimal(18,4),
  S_COMMENT varchar(101))
diststyle all;                                                              

create table lineitem (
  L_ORDERKEY bigint NOT NULL,
  L_PARTKEY bigint,
  L_SUPPKEY bigint,
  L_LINENUMBER integer NOT NULL,
  L_QUANTITY decimal(18,4),
  L_EXTENDEDPRICE decimal(18,4),
  L_DISCOUNT decimal(18,4),
  L_TAX decimal(18,4),
  L_RETURNFLAG varchar(1),
  L_LINESTATUS varchar(1),
  L_SHIPDATE date,
  L_COMMITDATE date,
  L_RECEIPTDATE date,
  L_SHIPINSTRUCT varchar(25),
  L_SHIPMODE varchar(10),
  L_COMMENT varchar(44))
distkey (L_ORDERKEY)
sortkey (L_RECEIPTDATE);

create table partsupp (
  PS_PARTKEY bigint NOT NULL,
  PS_SUPPKEY bigint NOT NULL,
  PS_AVAILQTY integer,
  PS_SUPPLYCOST decimal(18,4),
  PS_COMMENT varchar(199))
diststyle even;

## Data Loading

A COPY command loads large amounts of data much more efficiently than using INSERT statements, and stores the data more effectively as well. Use a single COPY command to load data for one table from multiple files. Amazon Redshift then automatically loads the data in parallel. For your convenience, the sample data you will use is available in a public Amazon S3 bucket. To ensure that Redshift performs a compression analysis, set the COMPUPDATE parameter to ON in your COPY commands.

In [ ]:
COPY region FROM 's3://redshift-immersionday-labs/data/region/region.tbl.lzo'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;

COPY nation FROM 's3://redshift-immersionday-labs/data/nation/nation.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;

copy customer from 's3://redshift-immersionday-labs/data/customer/customer.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;

copy orders from 's3://redshift-immersionday-labs/data/orders/orders.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;

copy part from 's3://redshift-immersionday-labs/data/part/part.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;

copy supplier from 's3://redshift-immersionday-labs/data/supplier/supplier.json' manifest
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;

copy lineitem from 's3://redshift-immersionday-labs/data/lineitem-part/'
iam_role default
region 'us-west-2' gzip delimiter '|' COMPUPDATE PRESET;

copy partsupp from 's3://redshift-immersionday-labs/data/partsupp/partsupp.tbl.'
iam_role default
region 'us-west-2' lzop delimiter '|' COMPUPDATE PRESET;

The estimated time to load the data is as follows, note you can check timing information on actions in the performance and query tabs on the redshift console:

```
REGION (5 rows) - 20s
NATION (25 rows) - 2s
CUSTOMER (15M rows) – 31s
ORDERS - (76M rows) - 13s
PART - (20M rows) - 34s
SUPPLIER - (1M rows) - 7s
LINEITEM - (303M rows) - 48s
PARTSUPPLIER - (80M rows) 12s
```

A few key takeaways from the above COPY statements:

- COMPUPDATE PRESET ON will assign compression using the Amazon Redshift best practices related to the data type of the column but without analyzing the data in the table.
- COPY for the REGION table points to a specfic file (region.tbl.lzo) while COPY for other tables point to a prefix to multiple files (lineitem.tbl.)
- COPY for the SUPPLIER table points a manifest file (supplier.json)

## Vacuum

Lets see how VACUUM DELETE reclaims table space after delete operation.

First, capture **tbl\_rows**(Total number of rows in the table. This value includes rows marked for deletion, but not yet vacuumed) and **estimated\_visible\_rows**(The estimated visible rows in the table. This value does not include rows marked for deletion) for the ORDERS table. Copy the following command and run it.

```sql
select "table", size, tbl_rows, estimated_visible_rows
from SVV_TABLE_INFO
where "table" = 'orders';
```

| table | size | tbl\_rows | estimated\_visible\_rows |
| --- | --- | --- | --- |
| orders | 6484 | 76000000 | 76000000 |

Next, delete rows from the ORDERS table. Copy the following command and run it.

```sql
delete orders where o_orderdate between '1997-01-01' and '1998-01-01';
```

Next, capture the tbl\_rows and estimated\_visible\_rows for ORDERS table after the deletion.

Copy the following command and run it. Notice that the tbl\_rows value hasn't changed, after deletion. This is because rows are marked for soft deletion, but VACUUM DELETE is not yet run to reclaim space.

```sql
select "table", size, tbl_rows, estimated_visible_rows
from SVV_TABLE_INFO
where "table" = 'orders';
```

| table | size | tbl\_rows | estimated\_visible\_rows |
| --- | --- | --- | --- |
| orders | 6356 | 76000000 | 64436860 |

Now, run the VACUUM DELETE command. Copy the following command and run it.

```sql
vacuum delete only orders;
```

Confirm that the VACUUM command reclaimed space by running the following query again and noting the tbl\_rows value has changed.

```sql
select "table", size, tbl_rows, estimated_visible_rows
from SVV_TABLE_INFO
where "table" = 'orders';
```

**orderdate** is the sort key on orders table. If you always load data into orders table where **orderdate** is the current date, since current date is forward incrementing, data will always be loaded in incremental sortkey(orderdate) order. Hence, in that case VACUUM SORT will not be needed for orders table.

If you need to run VACUUM SORT, you can still manually run it as shown below. Copy the following command and run it.

```sql
vacuum sort only orders;
```

In order to run vacuum recluster on orders, copy the following command and run it.

```sql
vacuum recluster orders;
```

In order to run vacuum recluster on orders table in boost mode, copy the following command and run it.

```sql
vacuum recluster orders boost;
```